# (18) df: KNN

**Motivation**: Device = ```cuda:2``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Fig save dir

In [2]:
from figures.imgs import plot_weights

fig_dir = pjoin(fig_base_dir, 'nips_may22')
os.makedirs(fig_dir, exist_ok=True)
print(os.listdir(fig_dir))

kws_fig = {
    'transparent': True,
    'bbox_inches': 'tight',
    'dpi': 600,
}

['nelbo_point.pdf']

## Device

In [3]:
device_idx = 2
device = f'cuda:{device_idx}'

from analysis.final import sort_fits, analyze_fits

## Load, make df

### Fit names

In [4]:
fits, fits_st, fits_etc = sort_fits()
len(fits), len(fits_st), len(fits_etc)

(460, 90, 527)

### df

In [5]:
def accept_fn(name):
    archi = name.split('-')[1]
    accept = (
        'MNIST' in name and
        archi == '<conv+b|conv+b>'
    )
    return accept


fits = sorted(filter(accept_fn, fits))
fits_etc = sorted(filter(accept_fn, fits_etc))

print(f"# fits: {len(fits)},  # fits_etc: {len(fits_etc)}")

# fits: 100,  # fits_etc: 90

In [6]:
%%time

kws = dict(device=device, analysis_mode='knn', knn_n_iter=500)
df = pd.concat([analyze_fits(fits, **kws), analyze_fits(fits_etc, **kws)])

df = df.sort_values(by=['type', 'seed', 'n_dims', 'iteration'])
df = df.reset_index(drop=True)

save_obj(
    obj=df,
    file_name='df_knn',
    save_dir=tmp_dir,
    verbose=True,
    mode='df',
)

100%|███████████████████████████████████████████| 90/90 [20:02<00:00, 13.36s/it]


[PROGRESS] 'df_knn.df' saved at
/home/hadi/Dropbox/git/jb-vae/tmp

CPU times: user 9h 8min 51s, sys: 38min 21s, total: 9h 47min 12s
Wall time: 41min 53s


In [10]:
df.shape

(190190, 14)

In [11]:
df['seed'].value_counts()

seed
1    38038
2    38038
3    38038
4    38038
5    38038
Name: count, dtype: int64

In [12]:
df.pivot_table(
    values='accuracy',
    index=['type', 'n_dims'],
    columns=['size'],
    aggfunc=np.mean,
)

size                          200       1000     5000
type              n_dims                             
categorical       5       0.578691  0.644058  0.68436
                  10      0.705222  0.799990  0.85348
                  20      0.745337  0.843714  0.88956
                  50      0.769964  0.879950  0.92040
                  100     0.753189  0.875537  0.92544
gaussian          5       0.778688  0.878503  0.92488
                  10      0.672993  0.813313  0.89068
                  20      0.626359  0.769354  0.85716
                  50      0.603619  0.746064  0.83696
                  100     0.635520  0.788724  0.87528
gaussian-exp      10      0.641520  0.783649  0.86340
                  50      0.694219  0.836499  0.90560
                  100     0.694618  0.845754  0.91792
gaussian-relu     10      0.694159  0.817101  0.87700
                  50      0.710364  0.844156  0.90436
                  100     0.756715  0.880772  0.93276
gaussian-softplus 10      0.670348  0.815668  0.89284
                  50      0.657376  0.805483  0.88716
gaussian-square   10      0.759270  0.862517  0.90772
                  50      0.750792  0.850974  0.89972
laplace           5       0.808443  0.895737  0.93424
                  10      0.757276  0.869298  0.92444
                  20      0.740963  0.858219  0.91880
                  50      0.709793  0.835885  0.90224
                  100     0.700627  0.829816  0.89584
laplace-exp       10      0.675328  0.796064  0.86580
                  50      0.645619  0.777287  0.85144
laplace-relu      10      0.712093  0.828142  0.88620
                  50      0.779626  0.886686  0.93300
laplace-softplus  10      0.727051  0.849775  0.91176
                  50      0.706619  0.837728  0.90532
laplace-square    10      0.777404  0.876207  0.92196
                  50      0.789752  0.869228  0.91540
poisson           5       0.767395  0.850912  0.89176
                  10      0.815274  0.918605  0.94636
                  20      0.821804  0.928331  0.95796
                  50      0.824504  0.926652  0.95732
                  100     0.806529  0.925338  0.95764

In [14]:
df.loc[df['n_dims'] == 10].pivot_table(
    values='accuracy',
    index=['type'],
    columns=['size'],
    aggfunc=np.mean,
)

size,200,1000,5000
type,,,
categorical,0.705222,0.799990,0.85348
gaussian,0.672993,0.813313,0.89068
gaussian-exp,0.641520,0.783649,0.86340
gaussian-relu,0.694159,0.817101,0.87700
gaussian-softplus,0.670348,0.815668,0.89284
gaussian-square,0.759270,0.862517,0.90772
laplace,0.757276,0.869298,0.92444
laplace-exp,0.675328,0.796064,0.86580
laplace-relu,0.712093,0.828142,0.88620


In [15]:
df.loc[df['n_dims'] == 100].pivot_table(
    values='accuracy',
    index=['type'],
    columns=['size'],
    aggfunc=np.mean,
)

size,200,1000,5000
type,,,
categorical,0.753189,0.875537,0.92544
gaussian,0.635520,0.788724,0.87528
gaussian-exp,0.694618,0.845754,0.91792
gaussian-relu,0.756715,0.880772,0.93276
laplace,0.700627,0.829816,0.89584
poisson,0.806529,0.925338,0.95764


In [21]:
x = df['iteration'].value_counts().to_dict()
x = dict(sorted(x.items()))
x

{0: 570,
 1: 380,
 2: 380,
 3: 380,
 4: 380,
 5: 380,
 6: 380,
 7: 380,
 8: 380,
 9: 380,
 10: 380,
 11: 380,
 12: 380,
 13: 380,
 14: 380,
 15: 380,
 16: 380,
 17: 380,
 18: 380,
 19: 380,
 20: 380,
 21: 380,
 22: 380,
 23: 380,
 24: 380,
 25: 380,
 26: 380,
 27: 380,
 28: 380,
 29: 380,
 30: 380,
 31: 380,
 32: 380,
 33: 380,
 34: 380,
 35: 380,
 36: 380,
 37: 380,
 38: 380,
 39: 380,
 40: 380,
 41: 380,
 42: 380,
 43: 380,
 44: 380,
 45: 380,
 46: 380,
 47: 380,
 48: 380,
 49: 380,
 50: 380,
 51: 380,
 52: 380,
 53: 380,
 54: 380,
 55: 380,
 56: 380,
 57: 380,
 58: 380,
 59: 380,
 60: 380,
 61: 380,
 62: 380,
 63: 380,
 64: 380,
 65: 380,
 66: 380,
 67: 380,
 68: 380,
 69: 380,
 70: 380,
 71: 380,
 72: 380,
 73: 380,
 74: 380,
 75: 380,
 76: 380,
 77: 380,
 78: 380,
 79: 380,
 80: 380,
 81: 380,
 82: 380,
 83: 380,
 84: 380,
 85: 380,
 86: 380,
 87: 380,
 88: 380,
 89: 380,
 90: 380,
 91: 380,
 92: 380,
 93: 380,
 94: 380,
 95: 380,
 96: 380,
 97: 380,
 98: 380,
 99: 380,
 100: 380,

In [22]:
sum(x.values())

190190